In [2]:
import spacy
nlp= spacy.load('en_core_web_lg')

In [6]:
doc= nlp("i am loving this place")

for token in doc:
    print(token.text, " | ", token.has_vector, token.is_oov)

i  |  True False
am  |  True False
loving  |  True False
this  |  True False
place  |  True False


In [10]:
token1= doc[0]
token1

token1.vector

array([  2.1387  ,  -2.8931  ,  -2.998   ,  -9.5886  , -14.661   ,
        -1.7485  ,  -0.31449 ,   5.6111  , -11.32    ,   9.2488  ,
         1.6306  ,   5.5915  ,  -8.7249  ,   4.9277  ,  -0.87152 ,
        -8.1548  ,   3.4968  ,  -4.2225  ,  -7.0104  , -10.979   ,
         7.3406  ,   4.8825  ,  -7.2914  ,  -0.70134 ,  -5.0973  ,
        -6.6847  ,   0.79904 ,  -0.062886,  -6.7023  ,   6.0051  ,
        -5.488   ,  -5.1686  ,  -6.2868  ,  15.347   ,  11.283   ,
         1.3869  ,  -5.458   ,   1.9095  ,   7.4756  ,  -3.045   ,
         0.79274 ,  -0.37955 ,   4.1467  , -10.019   ,  11.515   ,
         6.5432  ,  -9.0175  ,  -5.8828  ,  -4.6018  ,   5.1201  ,
         6.5793  ,  -8.355   ,  -3.6003  ,   1.0909  , -12.827   ,
        -2.4821  ,   1.8718  ,  10.511   ,   4.714   ,   2.4928  ,
        20.351   ,  -2.5769  , -14.688   ,  -3.3844  , -13.291   ,
        -7.2758  ,   2.9169  ,   9.5984  ,   3.7328  ,   6.6528  ,
        -5.8376  ,   9.8689  ,   1.8093  ,  -7.1582  ,  10.669

In [11]:
token1.vector.shape

(300,)

In [18]:
# comparing a word similarity with each word
doc1= nlp("i need help assistance")
doc2= nlp("assistance")

for token in doc1:
    print(token.similarity(doc2))


-0.08731684329893592
0.388751834824334
0.594193853014776
1.0


In [25]:
doc1= nlp("i need help")
doc2= nlp("i need assistance")

doc1.similarity(doc2)

0.9582953940189616

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([doc1.vector], [doc2.vector])

array([[0.9582954]], dtype=float32)

NEWS CLASSIFICATION

In [32]:
import pandas as pd

In [39]:
data= pd.read_json('resources/news_dataset.json')
data.head()

,text,category
0,"Larry Nassar Blames His Victims, Says He 'Was ...",CRIME
1,"Woman Beats Cancer, Dies Falling From Horse",CRIME
2,Vegas Taxpayers Could Spend A Record $750 Mill...,SPORTS
3,This Richard Sherman Interception Literally Sh...,SPORTS
4,7 Things That Could Totally Kill Weed Legaliza...,BUSINESS


In [40]:
data.category.value_counts()

CRIME       2500
SPORTS      2500
BUSINESS    2500
Name: category, dtype: int64

In [42]:
from sklearn.preprocessing import LabelEncoder
l= LabelEncoder()
data['category']= l.fit_transform(data['category'])
data.head()

,text,category
0,"Larry Nassar Blames His Victims, Says He 'Was ...",1
1,"Woman Beats Cancer, Dies Falling From Horse",1
2,Vegas Taxpayers Could Spend A Record $750 Mill...,2
3,This Richard Sherman Interception Literally Sh...,2
4,7 Things That Could Totally Kill Weed Legaliza...,0


In [43]:
import spacy
nlp= spacy.load('en_core_web_lg')

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

TFIDF METHOD WITH PREPROCESSING

In [58]:
clf= Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

In [59]:
x_train, x_test, y_train, y_test= train_test_split(data.text, data.category, random_state= 1010, test_size= 0.2, stratify= data.category)

In [60]:
clf.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()), ('nb', MultinomialNB())])

In [61]:
y_pred= clf.predict(x_test)

In [62]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91       500
           1       0.90      0.91      0.91       500
           2       0.95      0.88      0.91       500

    accuracy                           0.91      1500
   macro avg       0.91      0.91      0.91      1500
weighted avg       0.91      0.91      0.91      1500



PREPROCESSING

In [67]:
def preprocess(text):
    doc= nlp(text)
    
    filter= []
    for token in doc:
        if not token.is_stop and not token.is_punct:
            filter.append(token.lemma_)
    return ' '.join(filter)        

In [73]:
preprocess("I am a resident of india. bheem is the PM of this countries.")

'resident india bheem pm country'

In [74]:
data['text_pre']= data['text'].apply(preprocess)

In [76]:
data.head()

,text,category,text_pre
0,"Larry Nassar Blames His Victims, Says He 'Was ...",1,Larry Nassar blame victim say victimize newly ...
1,"Woman Beats Cancer, Dies Falling From Horse",1,woman Beats Cancer die fall horse
2,Vegas Taxpayers Could Spend A Record $750 Mill...,2,vegas taxpayer spend Record $ 750 million New ...
3,This Richard Sherman Interception Literally Sh...,2,Richard Sherman Interception literally shake W...
4,7 Things That Could Totally Kill Weed Legaliza...,0,7 thing totally kill Weed Legalization Buzz


TFIDF WITH PREPROCESSING

In [77]:
x_train, x_test, y_train, y_test= train_test_split(data.text_pre, data.category, test_size= 0.2, stratify= data.category, random_state= 1010)

In [78]:
clf= Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()), ('nb', MultinomialNB())])

In [79]:
y_pred= clf.predict(x_test)

In [80]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92       500
           1       0.88      0.95      0.92       500
           2       0.96      0.89      0.92       500

    accuracy                           0.92      1500
   macro avg       0.92      0.92      0.92      1500
weighted avg       0.92      0.92      0.92      1500



Preprocessing helps accuracy increased 

In [81]:
data.head()

,text,category,text_pre
0,"Larry Nassar Blames His Victims, Says He 'Was ...",1,Larry Nassar blame victim say victimize newly ...
1,"Woman Beats Cancer, Dies Falling From Horse",1,woman Beats Cancer die fall horse
2,Vegas Taxpayers Could Spend A Record $750 Mill...,2,vegas taxpayer spend Record $ 750 million New ...
3,This Richard Sherman Interception Literally Sh...,2,Richard Sherman Interception literally shake W...
4,7 Things That Could Totally Kill Weed Legaliza...,0,7 thing totally kill Weed Legalization Buzz


In [133]:
#i want to create vector for each text_pre entry 

# 1) make a function that convert a string to vector i.e word to vec make a new column
# 2) use stack function to make 2D x_test and x_train
# 3) use minmax scaler to handle -ve values

def word2vec(text):
    doc= nlp(text)
    return doc.vector


#make another column entry for vector
data['vector']= data['text_pre'].apply(word2vec)

In [134]:
data.head()

,text,category,text_pre,vector
0,"Larry Nassar Blames His Victims, Says He 'Was ...",1,Larry Nassar blame victim say victimize newly ...,"[-0.5585511, -0.29323253, -0.9253956, 0.189389..."
1,"Woman Beats Cancer, Dies Falling From Horse",1,woman Beats Cancer die fall horse,"[-0.73039824, -0.43196002, -1.2930516, -1.0628..."
2,Vegas Taxpayers Could Spend A Record $750 Mill...,2,vegas taxpayer spend Record $ 750 million New ...,"[-1.9413117, 0.121578515, -3.2996283, 1.511650..."
3,This Richard Sherman Interception Literally Sh...,2,Richard Sherman Interception literally shake W...,"[-1.4702771, -0.685319, 0.57398, -0.31135806, ..."
4,7 Things That Could Totally Kill Weed Legaliza...,0,7 thing totally kill Weed Legalization Buzz,"[-1.037173, -1.9495698, -1.7179357, 1.2975286,..."


In [135]:
x_train, x_test, y_train, y_test= train_test_split(data.vector, data.category, test_size= 0.2, stratify= data.category, random_state= 1010)

In [136]:
x_train

4959    [-0.09262099, 0.8324502, -1.875309, -0.1152760...
6747    [-0.7066472, 2.5146642, 2.103504, 2.519867, 4....
4553    [-0.2771612, -0.3354671, -2.50299, 0.74886405,...
5059    [-1.6744835, -1.0313778, -2.1614141, 0.1729021...
6952    [-0.70192295, -0.6899906, -1.0743976, 1.362338...
                              ...                        
1997    [0.7281106, -0.977128, 0.92114913, 0.8735767, ...
6006    [-0.04983789, 0.57454455, -1.6549667, 0.703606...
5252    [-1.2712216, -2.4240131, -0.21046638, 0.080976...
2594    [1.6618699, -0.53169376, -0.38821873, 0.634601...
2113    [-0.48358262, 0.8249327, -0.1580825, 1.6971024...
Name: vector, Length: 6000, dtype: object

In [137]:
data.isnull().sum()

text        0
category    0
text_pre    0
vector      0
dtype: int64

In [138]:
type(x_train)

pandas.core.series.Series

In [139]:
type(y_train)

pandas.core.series.Series

In [ ]:
# convert into 2D matrix
import numpy as np

x_train_2d= np.stack(x_train)
x_test_2d= np.stack(x_test)

In [141]:
x_train_2d

array([[-0.09262099,  0.8324502 , -1.875309  , ...,  0.69715655,
        -2.102335  ,  0.702706  ],
       [-0.7066472 ,  2.5146642 ,  2.103504  , ..., -4.561042  ,
         1.6280117 , -1.2523199 ],
       [-0.2771612 , -0.3354671 , -2.50299   , ..., -1.5236179 ,
        -0.54069585,  0.47268632],
       ...,
       [-1.2712216 , -2.4240131 , -0.21046638, ...,  0.67019147,
         1.617395  , -0.6414272 ],
       [ 1.6618699 , -0.53169376, -0.38821873, ..., -1.3562266 ,
         0.06693377,  1.5580144 ],
       [-0.48358262,  0.8249327 , -0.1580825 , ..., -1.1776694 ,
         0.4504114 ,  0.2140092 ]], dtype=float32)

In [144]:
# handle -ve values
from sklearn.preprocessing import MinMaxScaler
m= MinMaxScaler()

x_train_2d_s= m.fit_transform(x_train_2d)
x_test_2d_s= m.transform(x_test_2d)

In [145]:
model= MultinomialNB()
model.fit(x_train_2d_s, y_train)

MultinomialNB()

In [147]:
y_pred= model.predict(x_test_2d_s)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.82      0.81       500
           1       0.85      0.88      0.86       500
           2       0.83      0.79      0.81       500

    accuracy                           0.83      1500
   macro avg       0.83      0.83      0.83      1500
weighted avg       0.83      0.83      0.83      1500

